# 배열에 값 할당 예제
***
Size가 10인 numpy array에 0부터 9까지 할당

### Use CPU

In [1]:
import numpy as np

In [2]:
# Numpy array를 생성 (size=10, value=0)
data_arr = np.zeros(10, dtype=np.int)

# 반복문을 돌면서 값을 채움
for i in range(data_arr.size):
    data_arr[i] = i

# 출력
print(data_arr)

[0 1 2 3 4 5 6 7 8 9]


### Use GPU

In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import driver, compiler

In [4]:
# Numpy array를 생성 (size=10, value=0)
host_input_memory = np.zeros(10, dtype=np.int)

# GPU 메모리 할당
device_memory = cuda.mem_alloc(host_input_memory.nbytes)

# Host에서 Device로 메모리 복사
cuda.memcpy_htod(device_memory, host_input_memory)

In [5]:
# Kernel code
kernel_code = """
__global__ void assign(int* arry)
{
  int idx = threadIdx.x;
  arry[idx] = idx;
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

# Get kernel function
assign_func = mod.get_function("assign")

# Run
assign_func(device_memory, block=(10, 1, 1))

In [6]:
# Device에서 Host로 메모리 복사
host_output_memory = np.empty_like(host_input_memory)
cuda.memcpy_dtoh(host_output_memory, device_memory)

# GPU 메모리 해제
device_memory.free()

# 최종 값 출력
print(host_output_memory)

[0 1 2 3 4 5 6 7 8 9]


### Use GPU (간소화)
pycuda.driver.In, pycuda.driver.Out, pycuda.driver.InOut을 사용하여
GPU 메모리 할당, 메모리 복사, GPU 메모리 해제 부분을 생략할 수 있음

In [7]:
# Numpy array를 생성 (size=10, value=0)
data_arr = np.zeros(10, dtype=np.int)

# Run
assign_func(cuda.Out(data_arr), block=(10, 1, 1))

#출력
print(data_arr)

[0 1 2 3 4 5 6 7 8 9]
